In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, precision_score, recall_score, f1_score
from imblearn.under_sampling import RandomUnderSampler
from scipy.stats import uniform

In [2]:
# Function to Load Data
def load_data_from_directory(directory_path):
    data_frames = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory_path, file_name)
            df = pd.read_csv(file_path)
            data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

# Function to Preprocess Data
def hex_to_int(x):
    try:
        return int(str(x), 16)
    except ValueError:
        return 0  # Handle non-hexadecimal values

def preprocess_data(df):
    df['timestamp'] = pd.to_datetime(df['timestamp']).astype(np.int64) // 10**9
    df['arbitration_id'] = df['arbitration_id'].apply(hex_to_int)
    df['data_field'] = df['data_field'].apply(hex_to_int)
    return df

# Function to Extract Features and Labels
def extract_features_labels(df):
    X = df.drop(columns='attack')
    y = df['attack']
    return X, y



In [3]:
# Function to Train Logistic Regression Model with RandomizedSearchCV
def train_logistic_regression(X_train, y_train):
   # Apply Random Under Sampling for Handling Class Imbalance
    rus = RandomUnderSampler(random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)


    # Standardize the Features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_resampled)

    # Define the model
    log_reg = LogisticRegression(solver='liblinear')

    # Define parameter distribution
    param_dist = {
        'C': uniform(0.01, 10),  # Regularization strength
        'penalty': ['l1', 'l2']  # Regularization type
    }

    # Set up RandomizedSearchCV
    random_search = RandomizedSearchCV(
        log_reg, param_distributions=param_dist, n_iter=300, scoring='f1', n_jobs=-1, cv=5, random_state=42
    )

    # Record training start time
    start_time = time.time()
    
    # Perform the random search
    random_search.fit(X_train_scaled, y_train_resampled)

    # Calculate training time
    training_time = time.time() - start_time
    
    return random_search.best_estimator_, scaler, training_time

# Function to Test the Model
def test_model(model, scaler, X_test, y_test):
    # Standardize the Features
    X_test_scaled = scaler.transform(X_test)

    # Record testing start time
    start_time = time.time()
    
    # Predict on Test Data
    test_predictions = model.predict(X_test_scaled)

     # Calculate testing time
    testing_time = time.time() - start_time
    
    # Evaluate Test Performance
    conf_matrix_test = confusion_matrix(y_test, test_predictions)
    mcc_test = matthews_corrcoef(y_test, test_predictions)
    accuracy_test = accuracy_score(y_test, test_predictions)
    precision_test = precision_score(y_test, test_predictions, average='weighted', zero_division=0)
    recall_test = recall_score(y_test, test_predictions, average='weighted', zero_division=0)
    f1_test = f1_score(y_test, test_predictions,  average='weighted', zero_division=0)
    informedness_test = recall_test - (1 - recall_test)
    markedness_test = precision_test - (1 - precision_test)

    return {
        'conf_matrix': conf_matrix_test,
        'mcc': mcc_test,
        'accuracy': accuracy_test,
        'precision': precision_test,
        'recall': recall_test,
        'f1_score': f1_test,
        'informedness': informedness_test,
        'markedness': markedness_test,
        'testing_time': testing_time
    }



In [4]:
# Directories
train_directories = [
    './Documents/Research/can-train-and-test/set_01/train_01/'
]

test_directories = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/'
]

# Iterate through each training directory
for train_dir in train_directories:
    
    # Load and preprocess training data
    df_train = load_data_from_directory(train_dir)
    df_train = preprocess_data(df_train)
    X_train, y_train = extract_features_labels(df_train)

    # Train the Logistic Regression Model
    best_model, scaler, training_time = train_logistic_regression(X_train, y_train)

    # Corresponding test directories for each training set
    corresponding_tests = [td for td in test_directories if td.startswith(train_dir.split('/')[0])]
    
    # Test the model on each corresponding test set
    for test_dir in corresponding_tests:
        # print(f"\nTesting on data from: {test_dir}")
        
        # Load and preprocess testing data
        df_test = load_data_from_directory(test_dir)
        df_test = preprocess_data(df_test)
        X_test, y_test = extract_features_labels(df_test)

        # Test the Model
        test_metrics = test_model(best_model, scaler, X_test, y_test)

        # Print Testing Metrics
        print(f"\n===============================================================================")
        print(f"\nTesting Metrics (Train: {train_dir} | Test: {test_dir}):")
        print(f"\n-----------------------------------------------------------------------")
        print(f"\nTraining Time: {training_time:.2f} seconds")
        print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
        print(f"\n-----------------------------------------------------------------------")
        print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
        print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
        print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
        print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
        print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
        print(f"Informedness: {test_metrics['informedness']:.4f}")
        print(f"Markedness: {test_metrics['markedness']:.4f}")




Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_01/train_01/ | Test: ./Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 37.99 seconds
Testing Time: 0.13 seconds

-----------------------------------------------------------------------
Accuracy: 73.62%
Precision: 97.91%
Recall: 73.62%
F1-Score: 83.83%
Matthews Correlation Coefficient: 0.0163
Informedness: 0.4724
Markedness: 0.9582


Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_01/train_01/ | Test: ./Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 37.99 seconds
Testing Time: 0.14 seconds

-----------------------------------------------------------------------
Accuracy: 75.97%
Precision: 94.42%
Recall: 75.97%
F1-Score: 84.15%
Matthews Correlation Coef

In [5]:
# Directories
train_directories = [
    './Documents/Research/can-train-and-test/set_02/train_01/'
]

test_directories = [
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/'
]

# Iterate through each training directory
for train_dir in train_directories:
    
    # Load and preprocess training data
    df_train = load_data_from_directory(train_dir)
    df_train = preprocess_data(df_train)
    X_train, y_train = extract_features_labels(df_train)

    # Train the Logistic Regression Model
    best_model, scaler, training_time = train_logistic_regression(X_train, y_train)

    # Corresponding test directories for each training set
    corresponding_tests = [td for td in test_directories if td.startswith(train_dir.split('/')[0])]
    
    # Test the model on each corresponding test set
    for test_dir in corresponding_tests:
        # print(f"\nTesting on data from: {test_dir}")
        
        # Load and preprocess testing data
        df_test = load_data_from_directory(test_dir)
        df_test = preprocess_data(df_test)
        X_test, y_test = extract_features_labels(df_test)

        # Test the Model
        test_metrics = test_model(best_model, scaler, X_test, y_test)

        # Print Testing Metrics
        print(f"\n===============================================================================")
        print(f"\nTesting Metrics (Train: {train_dir} | Test: {test_dir}):")
        print(f"\n-----------------------------------------------------------------------")
        print(f"\nTraining Time: {training_time:.2f} seconds")
        print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
        print(f"\n-----------------------------------------------------------------------")
        print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
        print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
        print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
        print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
        print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
        print(f"Informedness: {test_metrics['informedness']:.4f}")
        print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_02/train_01/ | Test: ./Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 126.90 seconds
Testing Time: 0.28 seconds

-----------------------------------------------------------------------
Accuracy: 85.72%
Precision: 99.85%
Recall: 85.72%
F1-Score: 92.21%
Matthews Correlation Coefficient: 0.0486
Informedness: 0.7144
Markedness: 0.9970


Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_02/train_01/ | Test: ./Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 126.90 seconds
Testing Time: 0.16 seconds

-----------------------------------------------------------------------
Accuracy: 77.42%
Precision: 99.60%
Recall: 77.42%
F1-Score: 87.05%
Matthews Correlation Co

In [6]:
# Directories
train_directories = [
    './Documents/Research/can-train-and-test/set_03/train_01/'
]

test_directories = [
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/'
]

# Iterate through each training directory
for train_dir in train_directories:
    # Load and preprocess training data
    df_train = load_data_from_directory(train_dir)
    df_train = preprocess_data(df_train)
    X_train, y_train = extract_features_labels(df_train)

    # Train the Logistic Regression Model
    best_model, scaler, training_time = train_logistic_regression(X_train, y_train)

    # Corresponding test directories for each training set
    corresponding_tests = [td for td in test_directories if td.startswith(train_dir.split('/')[0])]
    
    # Test the model on each corresponding test set
    for test_dir in corresponding_tests:
        # print(f"\nTesting on data from: {test_dir}")
        
        # Load and preprocess testing data
        df_test = load_data_from_directory(test_dir)
        df_test = preprocess_data(df_test)
        X_test, y_test = extract_features_labels(df_test)

        # Test the Model
        test_metrics = test_model(best_model, scaler, X_test, y_test)

        # Print Testing Metrics
        print(f"\n===============================================================================")
        print(f"\nTesting Metrics (Train: {train_dir} | Test: {test_dir}):")
        print(f"\n-----------------------------------------------------------------------")
        print(f"\nTraining Time: {training_time:.2f} seconds")
        print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
        print(f"\n-----------------------------------------------------------------------")
        print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
        print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
        print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
        print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
        print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
        print(f"Informedness: {test_metrics['informedness']:.4f}")
        print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_03/train_01/ | Test: ./Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 550.95 seconds
Testing Time: 0.18 seconds

-----------------------------------------------------------------------
Accuracy: 67.97%
Precision: 97.24%
Recall: 67.97%
F1-Score: 79.25%
Matthews Correlation Coefficient: 0.0991
Informedness: 0.3593
Markedness: 0.9448


Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_03/train_01/ | Test: ./Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 550.95 seconds
Testing Time: 0.13 seconds

-----------------------------------------------------------------------
Accuracy: 68.23%
Precision: 96.13%
Recall: 68.23%
F1-Score: 79.17%
Matthews Correlation Co

In [7]:
# Directories
train_directories = [
    './Documents/Research/can-train-and-test/set_04/train_01/'
]

test_directories = [
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/'
]

# Iterate through each training directory
for train_dir in train_directories:
    
    # Load and preprocess training data
    df_train = load_data_from_directory(train_dir)
    df_train = preprocess_data(df_train)
    X_train, y_train = extract_features_labels(df_train)

    # Train the Logistic Regression Model
    best_model, scaler, training_time = train_logistic_regression(X_train, y_train)

    # Corresponding test directories for each training set
    corresponding_tests = [td for td in test_directories if td.startswith(train_dir.split('/')[0])]
    
    # Test the model on each corresponding test set
    for test_dir in corresponding_tests:
        # print(f"\nTesting on data from: {test_dir}")
        
        # Load and preprocess testing data
        df_test = load_data_from_directory(test_dir)
        df_test = preprocess_data(df_test)
        X_test, y_test = extract_features_labels(df_test)

        # Test the Model
        test_metrics = test_model(best_model, scaler, X_test, y_test)

        # Print Testing Metrics
        
        print(f"\n===============================================================================")
        print(f"\nTesting Metrics (Train: {train_dir} | Test: {test_dir}):")
        print(f"\n-----------------------------------------------------------------------")
        print(f"\nTraining Time: {training_time:.2f} seconds")
        print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
        print(f"\n-----------------------------------------------------------------------")
        print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
        print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
        print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
        print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
        print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
        print(f"Informedness: {test_metrics['informedness']:.4f}")
        print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_04/train_01/ | Test: ./Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 10.47 seconds
Testing Time: 0.13 seconds

-----------------------------------------------------------------------
Accuracy: 62.23%
Precision: 94.71%
Recall: 62.23%
F1-Score: 75.03%
Matthews Correlation Coefficient: -0.0930
Informedness: 0.2445
Markedness: 0.8943


Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_04/train_01/ | Test: ./Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 10.47 seconds
Testing Time: 0.28 seconds

-----------------------------------------------------------------------
Accuracy: 72.59%
Precision: 99.46%
Recall: 72.59%
F1-Score: 83.81%
Matthews Correlation Coe